## Test Real Entities

In [1]:
# NBVAL_IGNORE_OUTPUT

from timelink.notebooks import TimelinkNotebook

tlnb = TimelinkNotebook(db_name='rentitynb', drop_if_exists=True)
tlnb.print_info()


Timelink version: 1.1.27
Project name: test-project
Project home: /Users/jrc/develop/timelink-py-worktree/main/tests/timelink-home/projects/test-project
Database type: sqlite
Database name: rentitynb
Kleio image: timelinkserver/kleio-server
Kleio server token: 4zb74...
Kleio server URL: http://127.0.0.1:8088
Kleio server home: /Users/jrc/develop/timelink-py-worktree/main/tests/timelink-home/projects/test-project
Kleio server container: hopeful_kalam
Kleio version requested: latest
Kleio server version: 12.9.588 (2025-06-06 16:04:22)
SQLite directory: /Users/jrc/develop/timelink-py-worktree/main/tests/timelink-home/projects/test-project/database/sqlite
Database version: 6ccf1ef385a6
Call print_info(show_token=True) to show the Kleio Server token
Call print_info(show_password=True) to show the Postgres password
TimelinkNotebook(project_name=test-project, project_home=/Users/jrc/develop/timelink-py-worktree/main/tests/timelink-home/projects/test-project, db_type=sqlite, db_name=rentitynb,

In [2]:
from timelink.api.models import Person, Attribute
from timelink.api.models.rentity import REntity, Link, LinkStatus
# log sql from sqlalchemy to stdout
import logging
logging.basicConfig()
logging.getLogger('sqlalchemy.engine').setLevel(logging.ERROR)


with tlnb.db.session() as session:
    # Create a person
    person1: Person = Person(id='john-doe',name="John Doe", sex='m')
    person2: Person = Person(id='john-doe2',name="John Doe II", sex='m')
    session.add(person1)
    session.add(person2)
    person1.attributes.append(Attribute(id='p1-a1',the_type='age',the_value='30', the_date='2020-01-01'))
    person2.attributes.append(Attribute(id='p1-a2',the_type='age',the_value='31', the_date='2021-01-01'))
    session.commit()


In [3]:

with tlnb.db.session() as session:
    person1 = session.get(Person,'john-doe')
    person2 = session.get(Person,'john-doe2')
    print(person1)
    print(person2)

with tlnb.db.session() as session:
    # Create a real person
    rpjd = REntity.same_as('john-doe',
                                'john-doe2',
                                real_id='rp-john-doe',
                                user='tester',
                                status=LinkStatus.MANUAL,
                                session=session)
    session.commit()
    print(rpjd.to_kleio())

person$John Doe/m/id=john-doe
  attribute$age/30/2020-01-01
person$John Doe II/m/id=john-doe2
  attribute$age/31/2021-01-01
rentity$John Doe/id=rp-john-doe
  attribute$age/30/2020-01-01
  attribute$age/31/2021-01-01
